by gc2300

In [1]:
SQL_SOURCE = 'https://gc2300.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-5b32c6916989>, line 23)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',').columns

# Task 1

# Sort data by station_id

Only checking trips with duration <= 3 hours 

In [ ]:
q1 = '''
SELECT * FROM citibike
WHERE tripduration <= 3600*3
ORDER BY start_station_id
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q1)), sep=',')

sort data by trip duration

Only checking trips with duration <= 3 hours 

In [ ]:
q2 = '''
SELECT * FROM citibike
WHERE tripduration <= 3600*3

ORDER BY tripduration
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(q2)), sep=',')

# Only show the top 10 records

In [ ]:
q3 = '''
SELECT * FROM citibike
ORDER BY tripduration
LIMIT 10
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q3)),sep=',')

# Only show the last 10 trips

In [ ]:
q4 = '''
SELECT * FROM citibike
ORDER BY tripduration DESC
LIMIT 10
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q4)),sep=',')

# List all unique start_station_id values

In [ ]:
q5 = '''
SELECT DISTINCT start_station_id
FROM citibike
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(q5)),sep=',')

# Aggregation functions: 

Count the number of trips

In [ ]:
q6 = '''
SELECT COUNT(tripduration)
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q6)))

Aggregation functions: 

Find the average/min/max trip duration

In [ ]:
q7 = '''
SELECT AVG(tripduration) as avt, min(tripduration) as mint, max(tripduration) as maxt
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q7)))

# Task 2

# Selecting trips started on Feb-02-2015 only 

In [ ]:
q8 ='''
SELECT * FROM citibike
WHERE starttime >= 'Feb-02-2015'
AND   starttime < 'Feb-03-2015'
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q8)),sep=',')

# Selecting trips started on the weekends
 -What are average trip duration during weekends?

In [ ]:
q9 = '''
SELECT AVG(tripduration) FROM citibike
WHERE extract(DOW FROM starttime) IN (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q9)),sep=',')

Selecting trips started on the weekdays
 -What are average trip duration during weekdays?

In [ ]:
q10 = '''
SELECT AVG(tripduration) FROM citibike
WHERE extract(DOW FROM starttime) IN (1,2,3,4,5)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q10)),sep=',')

# Task 3

# Showing the list of start station locations 
    -in web mercator projection

In [ ]:
q11 = '''
SELECT 
    CDB_TransformToWebmercator(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
        )
    ) as the_geom_webmercator,
    MIN(cartodb_id) as cartodb_id
FROM citibike

GROUP BY
    start_station_latitude,
    start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q11)), sep=',')

# Showing the number of trips started per station 

In [ ]:
q12 = '''
SELECT 
    start_station_id,
    CDB_TransformToWebmercator(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
        )
    ) as the_geom_webmercator,
    COUNT(tripduration) as numtrips
FROM citibike

GROUP BY
    start_station_id,
    start_station_latitude,
    start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q12)),sep=',')

#  but only for stations within 500m of Time Square

In [ ]:
q13 = '''
SELECT 
    start_station_id,
    CDB_TransformToWebmercator(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
        )
    ) as the_geom_webmercator,
    COUNT(tripduration) as numtrips
FROM citibike

WHERE
    ST_DWithin(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
    )::geography,
        CDB_LatLng(40.7577,-73.9857)::geography,
        500)

GROUP BY
    start_station_id,
    start_station_latitude,
    start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q13)),sep=',')

# Task 4

Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!

In [ ]:
q14 = '''
SELECT 
    start_station_id,
    CDB_TransformToWebmercator(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
        )
    ) as the_geom_webmercator,
    COUNT(tripduration) as numtrips,
    AVG(tripduration) as avgt
FROM citibike

WHERE
    ST_DWithin(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
    )::geography,
        CDB_LatLng(40.7577,-73.9857)::geography,
        500)
AND
        extract (DOW from starttime) IN (0,6)


GROUP BY
    start_station_id,
    start_station_latitude,
    start_station_longitude
    
ORDER BY avgt DESC
LIMIT 1
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q14)),sep=',')

# EXTRA

 create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 

In [ ]:
q15 = '''
SELECT 
    start_station_id,
    end_station_id,
    ST_Makeline(
        CDB_TransformToWebmercator(CDB_LatLng(
            start_station_latitude,
            start_station_longitude
            )),
        CDB_TransformToWebmercator(CDB_LatLng(
            end_station_latitude,
            end_station_longitude
        ))
    ) as the_geom_webmercator,
        COUNT(tripduration) as numtrips

FROM citibike

WHERE
    tripduration < 7200
AND
    ST_DWithin(CDB_LatLng(
        start_station_latitude,
        start_station_longitude
    )::geography,
        CDB_LatLng(40.7577,-73.9857)::geography,
        500)    


GROUP BY
    start_station_id,
    start_station_latitude,
    start_station_longitude,
    end_station_id,
    end_station_latitude,
    end_station_longitude
    

'''
pd.read_csv(StringIO.StringIO(queryCartoDB(q15)),sep=',')

## ASH - Score 100. Extra CRedit 0